<a href="https://colab.research.google.com/github/shubhs42/DeepLearnCVBootCamp/blob/main/NLP_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 2**
## **Artificial Neural Network for Text Sentiment Classification**

Text sentiment classification using neural networks (NN) involves training a model to analyze and classify the sentiment expressed in text, such as positive, negative, or neutral. Neural networks, particularly recurrent networks (RNNs) or transformers (which will be explored subsequently), can capture the sequential nature of text data, making them well-suited for this task. By embedding words into dense vectors, a feed forward neural network can learn to associate patterns in word usage with sentiment labels. During training, the model adjusts its weights to minimize error, enabling it to generalize to unseen text, ultimately predicting sentiment with high accuracy.

====================================================================================================

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 1**
This code imports necessary libraries for text preprocessing, model building, and evaluation.
1)  It uses <blue>**pandas**</blue> for <green>**data handling**</green>.
2) <blue>**Matplotlib**</blue> for <green>**plotting**</green>.
3) <blue>**NLTK**</blue> for <green>**tokenizing**</green> and <green>**stemming**</green> text.
4) It includes <blue>**scikit-learn**</blue> for <green>**splitting data**</green> and <green>**evaluation (classification_report)**</green>.
5) <blue>**Torch**</blue> is used to <green>**define**</green>, <green>**train**</green>, and <green>**optimize**</green> neural networks.
6) <blue>**gensim**</blue> is used for <green>**dictionary creation**</green> and <green>**token mapping**</green>, helping to convert text into numerical format for model training.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from gensim import corpora
from sklearn.metrics import classification_report

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 2**
This code <green>**loads**</green> the <blue>**"yelp_reviews_subset_2.csv"**</blue> dataset downloaded from the Notion page.

In [ ]:
# TODO: Download and Load the "yelp_reviews_subset_2.csv" dataset uploaded on the notion page using pandas
df = None # Replace None with the correct code
df

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 3**
This code maps review star ratings to sentiment categories and visualizes the sentiment distribution.

1) The function <blue>**map_sentiment**</blue> converts star ratings into <green>**three categories**</green>: negative (-1 for stars ≤ 2), neutral (0 for 3 stars), and positive (1 for stars ≥ 4).
2) It applies this function to the <blue>**Labels**</blue> column in df, creating a new column <blue>**sentiment**</blue>.
3) The code then plots a bar chart showing the <green>**distribution of the sentiments**</green> using matplotlib and pandas, labeling the x-axis as "Sentiment" and the y-axis as "No. of rows in df".

In [ ]:
def map_sentiment(stars_received):
    if stars_received <= 2:
        return -1
    elif stars_received == 3:
        return 0
    else:
        return 1


# Mapping stars to sentiment into three categories
df['sentiment'] = [ map_sentiment(x) for x in df['Labels']]
# Plotting the sentiment distribution
plt.figure()
pd.value_counts(df['sentiment']).plot.bar(title="Sentiment distribution in df")
plt.xlabel("Sentiment")
plt.ylabel("No. of rows in df")
plt.show()

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 4**
This code tokenizes text and stems the words using NLTK tools.

1) It <green>**tokenizes**</green> each sentence in the dataframe <blue>**df['tokenized_text']**</blue> using <blue>**word_tokenize**</blue> from NLTK, applying <green>**list comprehension**</green> to store the tokens as a list.
2) Next, it uses the <blue>**Porter Stemmer**</blue> to <green>**stem**</green> each token in <blue>**df['tokenized_text']**</blue>, storing the stemmed words in <blue>**df['stemmed_tokens']**</blue> using <green>**list comprehension**</green>.

Both processes are applied to make the text more suitable for sentiment classification by reducing it to basic word forms.

In [ ]:
# TODO: Tokenize each sentence into word tokens and store them as a list in the dataframe
# Use List Comprehension
df['tokenized_text'] = None # Replace None with the correct code
print(df['tokenized_text'].head(10))

ps = PorterStemmer()
# TODO: Use the Porter Stemmer to find stem words of each word for all the words in df["tokenized_text"]
# Hint: Use List Comprehension
df['stemmed_tokens'] = None # Replace None with the correct code
df['stemmed_tokens'].head(10)

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 5**
This code splits the dataset into training and testing sets for model evaluation.
<blue>**split_train_test**</blue> function uses <blue>**train_test_split**</blue> from scikit-learn to divide the dataset into <green>**training (70%)**</green> and <green>**testing (30%)**</green> sets based on the tokenized_text, stemmed_tokens, and other columns.

In [ ]:
# Train Test Split Function
def split_train_test(df, test_size=0.3, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(df[["Text", "Labels", "tokenized_text", "stemmed_tokens"]],
                                                        df['sentiment'],
                                                        shuffle=shuffle_state,
                                                        test_size=test_size,
                                                        random_state=15)
    print("Value counts for Train sentiments")
    print(Y_train.value_counts())
    print("Value counts for Test sentiments")
    print(Y_test.value_counts())
    print(type(X_train))
    print(type(Y_train))
    X_train = X_train.reset_index()
    X_test = X_test.reset_index()
    Y_train = Y_train.to_frame()
    Y_train = Y_train.reset_index()
    Y_test = Y_test.to_frame()
    Y_test = Y_test.reset_index()
    print(X_train.head())
    return X_train, X_test, Y_train, Y_test

# Call the train_test_split
X_train, X_test, Y_train, Y_test = split_train_test(df)

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 6**
This code sets the <blue>**device**</blue> for running the model, either using <green>**GPU**</green> if available or falling back to <green>**CPU**</green>.


In [ ]:
# TODO: Set device (GPU if available, else CPU)
device = None # Replace None with the correct code
print("Device available for running: ")
print(device)

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 7**
This code defines a Feedforward Neural Network model with three fully connected layers using PyTorch.

1) <blue>**fc1**</blue>: First fully connected layer that takes the <green>**input dimension (input_dim)**</green> and maps it to a <green>**hidden dimension (hidden_dim)**</green>.
2) <blue>**fc2**</blue>: Second fully connected layer that maps the <green>**hidden dimension (hidden_dim)**</green> to another <green>**hidden dimension (hidden_dim)**</green>.
3) <blue>**fc3**</blue>: Final layer that maps the <green>**hidden dimension**</green> to the <green>**output dimension (output_dim)**</green>, which corresponds to the sentiment classification.

In [ ]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()

        # TODO: Add an fc1 layer as described below
        # Linear function 1: vocab_size --> hiiden_dim
        self.fc1 = None # Replace None with the correct code

        # Non-linearity 1
        self.relu1 = nn.ReLU()

        # TODO: Add an fc2 layer as described below
        # Linear function 2: hidden_dim --> hidden_dim
        self.fc2 = None # Replace None with the correct code

        # Non-linearity 2
        self.relu2 = nn.ReLU()

        # TODO: Add an fc3 layer as described below
        # Linear function 3 (readout): hidden_dim --> output_dim
        self.fc3 = None  # Replace None with the correct code

    def forward(self, x):

        # TODO: Forward pass through fully connected layer 1
        out = None # Replace None with the correct code

        # Non-linearity 1
        out = self.relu1(out)

        # TODO: Forward pass through fully connected layer 2
        out = None # Replace None with the correct code

        # Non-linearity 2
        out = self.relu2(out)

        # TODO: Forward pass through fully connected layer 3
        out = None # Replace None with the correct code

        return F.softmax(out, dim=1)

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 8**
This code defines a function to create a dictionary of tokens from a DataFrame using Gensim's corpora.Dictionary.

1) <blue>**make_dict**</blue> generates a <green>**token dictionary**</green> from the <blue>**stemmed_tokens**</blue> column in the DataFrame.
2) If <blue>**padding=True**</blue>, it adds a <green>**padding token ('pad')**</green> to the dictionary, which is useful for models that require fixed input sizes.
3) If <blue>**padding=False**</blue>, the dictionary is created directly from the tokenized text <green>**without adding a padding token**</green>.
4) The <blue>**review_dict**</blue> variable stores the dictionary, and in this case, it is created <green>**without padding**</green> by calling the function with <blue>**padding=False**</blue>.

This process is helpful for converting text tokens into numerical indices that can be used as input for machine learning models.

In [ ]:
# Function to return the dictionary either with padding word or without padding
def make_dict(top_data_df_small, padding=True):
    if padding:
        print("Dictionary with padded token added")
        review_dict = corpora.Dictionary([['pad']])
        review_dict.add_documents(top_data_df_small['stemmed_tokens'])
    else:
        print("Dictionary without padding")
        review_dict = corpora.Dictionary(top_data_df_small['stemmed_tokens'])
    return review_dict

# Make the dictionary without padding for the basic models
review_dict = make_dict(df, padding=False)

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 9**
This code creates functions to generate input and target tensors for the neural network.

1) <blue>**make_bow_vector**</blue>: This function creates a <green>**bag-of-words (BOW)**</green> vector from a tokenized sentence.

2) It initializes a <blue>**zero vector**</blue> of size <blue>**VOCAB_SIZE (30,056)**</blue>, where <green>**each index**</green> represents a word from the vocabulary.

3) <blue>**make_target**</blue>: This function converts a <green>**sentiment label (-1, 0, or 1)**</green> into a tensor for the output.

4) <blue>**Negative sentiment (-1)**</blue> maps to <green>**0**</green>, <blue>**neutral sentiment (0)**</blue> maps to <green>**1**</green>, and <blue>**positive sentiment (1)**</blue> maps to <green>**2**</green>.

In [ ]:
VOCAB_SIZE = 30056
NUM_LABELS = 3

# Function to make bow vector to be used as input to network
def make_bow_vector(review_dict, sentence):
    vec = torch.zeros(VOCAB_SIZE, dtype=torch.float64, device=device)
    for word in sentence:
        vec[review_dict.token2id[word]] += 1
    return vec.view(1, -1).float()

# Function to get the output tensor
def make_target(label):
    if label == -1:
        return torch.tensor([0], dtype=torch.long, device=device)
    elif label == 0:
        return torch.tensor([1], dtype=torch.long, device=device)
    else:
        return torch.tensor([2], dtype=torch.long, device=device)

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 10**
1) <blue>**Model Initialization**</blue>: The <green>**Feedforward Neural Network**</green> is instantiated with input_dim, hidden_dim, and output_dim.
2) <blue>**Device Transfer**</blue>: The model is moved to the <green>**GPU**</green> or <green>**CPU**</green> as specified earlier.
3) <blue>**Loss Function**</blue>: The loss function is set to <green>**CrossEntropyLoss**</green>, suitable for multi-class classification.
4) <blue>**Optimizer**</blue>: The optimizer is <green>**SGD (Stochastic Gradient Descent)**</green> with a <green>**learning rate**</green> of <green>**1e-3**</green>.

In [ ]:
VOCAB_SIZE = len(review_dict)

input_dim = VOCAB_SIZE
hidden_dim = 500
output_dim = 3
num_epochs = 100

# TODO: Call the FeedForwardNeuralNetwork Class object with appropriate arguements
ff_nn_bow_model = None # Replace None with the correct code

# TODO: Move the model to device
ff_nn_bow_model = None # Replace None with the correct code

# TODO: Define the loss function as Cross Entropy Loss
loss_function = None # Replace None with the correct code

# TODO: Define a Stochastic Gradient Descent Optimizer with learning rate of 1e-3
optimizer = None # Replace None with the correct code

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 11**
1) <blue>**Bag-of-Words Vector**</blue>: The <green>**make_bow_vector**</green> function is used to create a BOW vector from the stemmed tokens.
2) <blue>**Forward Pass**</blue>: The <green>**BOW vector**</green> is passed through the <green>**feedforward neural network**</green> to get the output probabilities.
3) <blue>**Loss Calculation**</blue>: The <green>**CrossEntropyLoss**</green> is computed between the output probabilities and the target label.
4) <blue>**Gradient Update**</blue>: <green>**Gradients**</green> are calculated and the parameters are updated using backpropagation and <green>**optimizer.step()**</green>.
5) <blue>**Loss Logging**</blue>: The <green>**average loss per epoch**</green> is written to the loss file for tracking training progress.

In [ ]:
# Open the file for writing loss
ffnn_loss_file_name = 'ffnn_bow_class_big_loss_500_epoch_100_less_lr.csv'
f = open(ffnn_loss_file_name,'w')
f.write('iter, loss')
f.write('\n')
losses = []
iter = 0
# Start training
for epoch in range(num_epochs):
    if (epoch+1) % 25 == 0:
        print("Epoch completed: " + str(epoch+1))
    train_loss = 0
    for index, row in X_train.iterrows():
        # Clearing the accumulated gradients
        optimizer.zero_grad()

        # TODO: Make the bag of words vector for stemmed tokens
        bow_vec = None # Replace None with the correct code

        # TODO: Forward pass to get output
        probs = None # Replace None with the correct code

        # Get the target label
        target = make_target(Y_train['sentiment'][index])

        # TODO: Calculate Loss: softmax --> cross entropy loss
        loss = None # Replace None with the correct code

        # Accumulating the loss over time
        train_loss += loss.item()

        # Getting gradients w.r.t. parameters
        loss.backward()

        # Updating parameters
        optimizer.step()
    f.write(str((epoch+1)) + "," + str(train_loss / len(X_train)))
    f.write('\n')
    train_loss = 0

f.close()

<style>
blue {
  color: skyblue;
}

green {
  color: lightgreen;
}
</style>

### **Step - 12**
This code evaluates the performance of the trained feedforward neural network on the test dataset and visualizes the loss during training.

1) <blue>**Prediction Loop**</blue>: It iterates through the test set X_test without tracking gradients <green>**(using torch.no_grad())**</green> for efficient inference. For each test sample, it generates a <green>**BOW vector**</green> and passes it through the model to obtain predicted probabilities.

2) <blue>**Classification Report**</blue>: The <green>**classification_report**</green> from <green>**sklearn.metrics**</green> is printed to evaluate the model's performance, displaying precision, recall, F1-score, and support for each class.

3) <blue>**Loss DataFrame**</blue>: The loss data saved in <green>**ffnn_loss_file_name**</green> is read into a DataFrame.

4) <blue>**Loss Plotting**</blue>: The loss is plotted using pandas and saved as a JPEG file named <green>**"ffnn_bow_loss_500_padding_100_epochs_less_lr.jpg"**</green>.

In [ ]:
bow_ff_nn_predictions = []
original_lables_ff_bow = []

with torch.no_grad():
    for index, row in X_test.iterrows():
        bow_vec = make_bow_vector(review_dict, row['stemmed_tokens'])
        probs = ff_nn_bow_model(bow_vec)
        bow_ff_nn_predictions.append(torch.argmax(probs, dim=1).cpu().numpy()[0])
        original_lables_ff_bow.append(make_target(Y_test['sentiment'][index]).cpu().numpy()[0])

print(classification_report(original_lables_ff_bow,bow_ff_nn_predictions))
ffnn_loss_df = pd.read_csv(ffnn_loss_file_name)
print(len(ffnn_loss_df))
print(ffnn_loss_df.columns)
ffnn_plt_500_padding_100_epochs = ffnn_loss_df[' loss'].plot()
fig = ffnn_plt_500_padding_100_epochs.get_figure()
fig.savefig("ffnn_bow_loss_500_padding_100_epochs_less_lr.jpg")


### Advanced Steps (Optional for people targeting basic level)

### **Step - 13**
This code defines and trains a Feedforward Neural Network model with Dropout for regularization.

1. **Model Initialization**: The **Feedforward Neural Network** is instantiated with input_dim, hidden_dim, output_dim, and a dropout rate.
2. **Device Transfer**: The model is moved to the **GPU** or **CPU**.
3. **Loss Function**: The loss function is set to **CrossEntropyLoss**.
4. **Optimizer**: The optimizer is **SGD (Stochastic Gradient Descent)** with a **learning rate** of **1e-3**.
5. **Training Loop**: The model is trained on the training data, calculating and minimizing the loss using backpropagation and the optimizer.
6. **Loss Logging**: The average loss per epoch is written to a new loss file for tracking training progress.

In [ ]:
# Initialize the Feedforward Neural Network model with dropout
# TODO: Create a model instance with the given input, hidden, and output dimensions
ff_nn_bow_model_dropout = None  # Replace None with model initialization (include dropout_rate parameter)
ff_nn_bow_model_dropout.to(device)

# TODO: Define loss function and optimizer
loss_function_dropout = None  # Replace None with appropriate loss function
optimizer_dropout = None  # Replace None with optimizer

# TODO: Prepare to record training loss
ffnn_loss_file_name_dropout = 'ffnn_bow_class_big_loss_500_epoch_100_less_lr_dropout.csv'
f_dropout = open(ffnn_loss_file_name_dropout, 'w')
f_dropout.write('iter, loss\n')
losses_dropout = []
iter_dropout = 0

# Train the model
ff_nn_bow_model_dropout.train()
for epoch in range(num_epochs):
    if (epoch + 1) % 25 == 0:
        print(f"Epoch completed: {epoch + 1}")

    train_loss_dropout = 0

    # TODO: Iterate through training samples
    for index, row in X_train.iterrows():
        optimizer_dropout.zero_grad()

        # TODO: Create bag-of-words input vector and target
        bow_vec = None  # Replace None with code to create BoW vector from review_dict and tokens (call previous functions defined)
        probs = None  # Replace None with forward pass through the model
        target = None  # Replace None with code to create target tensor

        # TODO: Compute loss
        loss = None  # Replace None with loss computation
        train_loss_dropout += loss.item()

        # Backward pass and optimization
        loss.backward()
        optimizer_dropout.step()

    # Log average loss for the epoch
    f_dropout.write(f"{epoch + 1},{train_loss_dropout / len(X_train)}\n")
    train_loss_dropout = 0

f_dropout.close()


### **Step - 14**
This code evaluates the performance of the trained feedforward neural network with Dropout on the test dataset and visualizes the loss during training, comparing it with the model without dropout.

1. **Prediction Loop**: It iterates through the test set X_test with the dropout model.
2. **Classification Report**: The **classification_report** from **sklearn.metrics** is printed to evaluate the model's performance with dropout.
3. **Loss DataFrame**: The loss data saved for both models is read into DataFrames.
4. **Loss Plotting**: The loss for both models is plotted on the same graph for comparison and saved as a JPEG file.

In [ ]:
bow_ff_nn_predictions_dropout = []
original_lables_ff_bow_dropout = []

ff_nn_bow_model_dropout.eval() # Set model to evaluation mode
with torch.no_grad():
    for index, row in X_test.iterrows():
        bow_vec = make_bow_vector(review_dict, row['stemmed_tokens'])
        probs = ff_nn_bow_model_dropout(bow_vec)
        bow_ff_nn_predictions_dropout.append(torch.argmax(probs, dim=1).cpu().numpy()[0])
        original_lables_ff_bow_dropout.append(make_target(Y_test['sentiment'][index]).cpu().numpy()[0])

print("Classification Report with Dropout:")
print(classification_report(original_lables_ff_bow_dropout, bow_ff_nn_predictions_dropout))

ffnn_loss_df = pd.read_csv(ffnn_loss_file_name)
ffnn_loss_df_dropout = pd.read_csv(ffnn_loss_file_name_dropout)

plt.figure(figsize=(10, 6))
plt.plot(ffnn_loss_df['iter'], ffnn_loss_df[' loss'], label='Without Dropout')
plt.plot(ffnn_loss_df_dropout['iter'], ffnn_loss_df_dropout[' loss'], label='With Dropout')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss with and Without Dropout')
plt.legend()
plt.grid(True)
plt.savefig("ffnn_bow_loss_comparison_dropout.jpg")
plt.show()